This is a notebook that calculates neutron capture cross sections at a given T9 in units of millibarns $(mb)$

Here we import the relavent python packages

In [ ]:
import sys
!{sys.executable} -m pip install --quiet wnutils

import sys
!{sys.executable} -m pip install --quiet astropy

import os, io, requests
from lxml import etree

import wnutils.xml as wx
import numpy as np
from astropy import constants as pc
from astropy import units as u
from matplotlib import pyplot as plt

from collections import defaultdict

Now we download an xml that contains the reaction data

In [ ]:
my_xml = wx.Xml(io.BytesIO(requests.get('https://osf.io/vqhxr/download').content))

In [ ]:
reac = my_xml.get_reaction_data("[reactant = 'n' and count(reactant) = 2  and product = 'gamma' and count(product) = 2]")

In [ ]:
t9 = 0.8
T = t9 * 1.e9 * u.K

In [ ]:
v_t = np.sqrt((2 * pc.k_B.decompose(u.cgs.bases) * T) / pc.m_n.decompose(u.cgs.bases))

In [ ]:
sig_dict = {}

filter = {}

In [ ]:
for s in reac:
    reaction = reac[s]
    rate = reaction.compute_rate(t9) * u.cm**3 /u.second /u.mol 
    sigma = (rate / (pc.N_A * v_t)).to(u.barn)
        
    usigma = 1000 * sigma / u.barn
    sig_dict[s] = usigma 

In [ ]:
nuc_xpath = str("[z = 43 and a >= 90 and a <= 100]")

nuclides = {}

nuc = my_xml.get_nuclide_data(nuc_xpath)
nuclides.update(nuc)

We now specify that we only want $(n,\gamma)$ reactions

In [ ]:
for reaction in reac:
    for reactant in reac[reaction].reactants:
        if reactant in nuclides:
            filter[reaction] = nuclides[reactant]['n']


In [ ]:
for key in list(sig_dict.keys()):
    if key not in filter:
        del sig_dict[key]

In [ ]:
print('Cross Sections at T9 of ' + str(t9))
# change 2 in {.2f} to change number of decimal places shown
for k,v in sorted(sig_dict.items()):
    x ="{:.2f}".format(v)
    print(str(k)+': ' + str(x) + ' mb')

We now plot $\sigma_n$ vs $n$

In [ ]:
combine = defaultdict(list)

for d in (sig_dict, filter):
    for key, value in d.items():
       combine[key].append(value)
    
sortedcombine = {k: v for k, v in sorted(combine.items(), key=lambda item: item[1][1])}

x = []
y = []

for s in list(sortedcombine.values()):
    x.append(s[1])
    y.append(s[0])

#print(sortedcombine)
#print(x)
#print(y)

plt.plot(x, y)
plt.ylabel('$\\sigma_n$')
plt.xlabel('n')
plt.show()